## Código contendo o modelo em Python com PuLP

In [25]:
import pulp

In [26]:
# Criar o modelo de otimização (Minimização)
model = pulp.LpProblem("Montagem_de_Rodas", pulp.LpMinimize)

In [27]:
# Variáveis que representam a quantidade de cada tipo de roda
x1 = pulp.LpVariable("41P", lowBound=0, cat="Integer")
x2 = pulp.LpVariable("C0V", lowBound=0, cat="Integer")
x3 = pulp.LpVariable("CR4", lowBound=0, cat="Integer")

In [28]:
#Calculo das demandas mínimas para um dia específico baseado na demandas minimas da semana anterior

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')



def calcular_demanda_minima_7dias(serie_rodas, nome_roda):

    if serie_rodas.empty:
        print(f"Sem dados para {nome_roda}.")
        return None

    # garantir que o índice é datetime (caso seja date puro)
    idx = pd.to_datetime(serie_rodas.index)

    ultima_data = idx.max()
    data_inicio = ultima_data - pd.Timedelta(days=6)

    # filtrar janela dos últimos 7 dias
    mask = (idx >= data_inicio) & (idx <= ultima_data)
    serie_7dias = serie_rodas[mask]

    if serie_7dias.empty:
        print(f"Não há dados nos últimos 7 dias para {nome_roda}.")
        return None

    demanda_min = serie_7dias.min()

    print(f"Demanda mínima (janela de 7 dias) - {nome_roda}: {demanda_min} unidades/dia")
    print(f"Período considerado: {data_inicio.date()} até {ultima_data.date()}\n")

    return demanda_min


# Leitura do CSV
df = pd.read_csv('/content/inteli.csv')

# Filtragem por tipo de roda
df_41P = df[df['PR'].str.contains('RAD:41P', na=False)].copy()
df_C0V = df[df['PR'].str.contains('RAD:C0V', na=False)].copy()
df_CR4 = df[df['PR'].str.contains('RAD:CR4', na=False)].copy()


# Calcula demanda minima para roda tipo 41P

df_41P['M310'] = pd.to_datetime(df_41P['M310'],
                                format='%Y-%m-%d-%H.%M.%S.%f',
                                errors='coerce')
df_41P['dia'] = df_41P['M310'].dt.date
serie_rodas_41P = df_41P.groupby('dia').size() * 4

x1_min = calcular_demanda_minima_7dias(serie_rodas_41P, "41P")


# Calcula demanda minima para roda do tipo C0V

df_C0V['M310'] = pd.to_datetime(df_C0V['M310'],
                                format='%Y-%m-%d-%H.%M.%S.%f',
                                errors='coerce')
df_C0V['dia'] = df_C0V['M310'].dt.date
serie_rodas_C0V = df_C0V.groupby('dia').size() * 4

x2_min = calcular_demanda_minima_7dias(serie_rodas_C0V, "C0V")


# Calcula demanda minima para roda do tipo CR4

df_CR4['M310'] = pd.to_datetime(df_CR4['M310'],
                                format='%Y-%m-%d-%H.%M.%S.%f',
                                errors='coerce')
df_CR4['dia'] = df_CR4['M310'].dt.date
serie_rodas_CR4 = df_CR4.groupby('dia').size() * 4

x3_min = calcular_demanda_minima_7dias(serie_rodas_CR4, "CR4")



Demanda mínima (janela de 7 dias) - 41P: 16 unidades/dia
Período considerado: 2025-09-09 até 2025-09-15

Demanda mínima (janela de 7 dias) - C0V: 36 unidades/dia
Período considerado: 2025-09-09 até 2025-09-15

Demanda mínima (janela de 7 dias) - CR4: 32 unidades/dia
Período considerado: 2025-09-09 até 2025-09-15



In [29]:
# variável que representa estoque inicial do dia, com o que sobrou do estoque anterior
# Nesse caso, vamos simular que o estoque incial de um dia

estoque_inical = 24

In [30]:
# Função objetivo: Minimizar estoque
model += x1 + x2 + x3, "Estoque mínimo"

In [32]:
# Restrições

# Restrição de estoque
model += x1 + x2 + x3 + estoque_inical <= 4096, "Estoque_max"

# Restições por demanda
model += x1 >= x1_min, "Demanda_min_x1"
model += x1 <= 136, "Demanda_max_x1"

model += x2 >= x2_min, "Demanda_min_x2"
model += x2 <= 128, "Demanda_max_x2"

model += x3 >= x3_min, "Demanda_min_x3"
model += x3 <= 168, "Demanda_max_x3"

# Restiçoões de não negatividade já são lidadas em lowBound=0 quando definimos LpVariable

In [33]:
model.solve()

1

In [34]:
# Resultados
print("Status:", pulp.LpStatus[model.status])
print("Rodas de Liga Leve (x1):", x1.value())
print("Rodas de Aço (x2):", x2.value())
print("Rodas de Esportiva (x3):", x3.value())
print("Estoque mínimo:", pulp.value(model.objective))

Status: Optimal
Rodas de Liga Leve (x1): 16.0
Rodas de Aço (x2): 36.0
Rodas de Esportiva (x3): 32.0
Estoque mínimo: 84.0
